In [1]:
import requests as r
from bs4 import BeautifulSoup
import re
from collections import Counter
import threading
import time


In [2]:
def getLink(page): #把搜尋頁丟近來抓連結的function
    print('thread'+str(page)+'start') #測試多執行緒有沒有在動
    res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
    soup = BeautifulSoup(res.text,'lxml')
    links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
    print(len(links))
    global alinks #拿到外面alinks的值如果沒有加global會變成自己增加區域變數
    alinks += links #alinks = alinks + links 

#     print('thread '+str(page)+' end ')    

In [3]:
class getLinkThread (threading.Thread): #多線程處理
    def __init__(self,page): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self) #繼承父類別（照著打就好）
        self.page=page #this.page=page
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getLink(self.page) #將要爬的頁數丟給getLink去執行


In [4]:
def getWord(link): #把網址丟進來提取文字後做統計的function
    print(link)
    res = r.get(link)
    soup = BeautifulSoup(res.text,'lxml')
    list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
    list2 = str(soup.select('div.content')).upper() #內文
    list3 = list1 + list2 
    words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字
#     time.sleep(0.2)

    for word in words: # 計算所有英文單字的數量
        global wc
        if word in wc:
            wc[word] += 1
        else:
            wc[word] = 1    

In [5]:
class getWordThread (threading.Thread): #多線程處理
    def __init__(self,link): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self)
        self.link=link
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getWord(self.link)


In [6]:
#第一頁開始：
url='https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&jobcat=2007000000&order=2&asc=0&page=1'
res = r.get(url)
soup = BeautifulSoup(res.text,'lxml')
links = soup.select('div.jobname_summary.job_name > a') #裡面是沒有HOST的
HOST = 'https://www.104.com.tw'
link_list = [HOST + link['href'] for link in links] #把裡面每一個網址都加上HOST
#把link全加到link_list裡面


# 換頁：
buttons = soup.select('span > a')
first_page = 1
# int(buttons[3]['onclick'].split('gopage(')[1].split(');return')[0]) - 2 #3-2
#顯示當前頁面＋當頁所以職缺(一次爬?頁)
#把當頁所有職缺的網址放到alink裡面
alinks = []
page_to_crawl = 10  #要爬幾頁
threads=[]
for page in range(1,150):
    Thread=getLinkThread(page)
    threads.append(Thread)    
for i in threads:
    i.start() #執行緒開始
#     time.sleep(1)
for i in threads:
    i.join()
    
#     print("current page {}".format("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page=".format(page)+"&psl=N_A"))
#     res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
#     soup = BeautifulSoup(res.text,'lxml')
#     links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
#     print(len(links))
#     alinks += links #alinks = alinks + links
#     time.sleep(0.5)


# for i in range(1,10+1): 
#     Thread=MyClass(i) 
#     threads.append(Thread)
# for thread in threads:
#     thread.start() #執行緒開始
# for thread in threads:
#     thread.join()#等同於java中的join 所有執行序跑完在繼續執行下一行指令
#     getLink(page)



#小整合：
wc = Counter() # word count   wc是k:v的形式-> ‘MYSQL': 4’
result_dict = {}
threadwords=[]
for link in alinks:
    threadword=getWordThread(link) 
    threadwords.append(threadword)
for i in threadwords:
    i.start() #執行緒開始
#     time.sleep(0.5)
for i in threadwords:
    i.join()
    
#         res = r.get(link)
#         soup = BeautifulSoup(res.text,'lxml')
#         list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
#         list2 = str(soup.select('div.content')).upper() #內文
#         list3 = list1 + list2
#         words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字

#         for word in words: # 計算所有英文單字的數量
#             if word in wc:
#                 wc[word] += 1
#             else:
#                 wc[word] = 1


#     continue
#     print(link) #有一些廣告或外包網continue
        
string = 'C|C++|C#|PYTHON|JAVA|JAVASCRIPT|PHP|HTML|SQL|CSS|R|BASH|RUBY|PERL|SCALA|SWIFT|GO|DELPHI|TYPESCRIPT|MYSQL'
langs = string.split('|') #string裡面的語言切開

for lang in langs: #每一個語言
    if lang in wc.keys(): #如果有在wc的key裡的話
        result_dict[lang] = wc[lang] #lang是key ;wc[lang]語言數量是value

result_dict

thread1startthread2start

thread3startthread4start

thread5start
thread6start
thread7start
thread9start
thread8start
thread12start
thread10start
thread11start
thread13startthread15startthread14start

thread16start

thread17start
thread18start
thread19start
thread20start
thread21startthread22start
thread23start
thread25start
thread26start

thread27start
thread24start
thread28start
thread29start
thread30start
thread31start
thread32start
thread33start
thread34start
thread35start
thread37start
thread38start
thread39start
thread36start
thread40start
thread41start
thread42start
thread43start
thread44start
thread46start
thread45start
thread47start
thread49startthread48start

thread50startthread51start

thread52start
thread53start
thread54startthread56startthread55startthread57startthread58startthread59startthread60start






thread62startthread61startthread63start


thread64start
thread65start
thread66startthread67start

thread68start
thread69start
thread70start
thread71start
thread72startth

Exception in thread Thread-220:
Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/sk

https://www.104.com.tw/job/?jobno=4l2a2&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4xgjh&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5g749&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=46f72&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5fwr0&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4jy3z&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=5gm8l&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4yrvg&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ijhh&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ijhi&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ijh4&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=3ijh5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=2x2o4&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=2x2o7&jobsource=104_bank1&hotjo

Exception in thread Thread-269:
Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/sk

https://www.104.com.twhttp://hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=104_bank1&urlpage=job_description&jobno=9173564https://www.104.com.twhttp://hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=104_bank1&urlpage=job_description&jobno=9173554https://www.104.com.tw/job/?jobno=3w80f&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=59sqe&jobsource=104_bank1&hotjob_chr=





https://www.104.com.tw/job/?jobno=3w80h&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=3w80j&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3wdmc&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3xbum&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=41n6j&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3fy26&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3lc5g&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=tgj2&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/

...

https://www.104.com.tw/job/?jobno=5c0je&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=52kkz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5a89f&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4tpd4&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4x7id&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4h20w&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=482v1&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3r2fn&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=269kn&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5aco0&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5aer8&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=2nrgi&jobsource=104_bank1&hotjob_chr=




https://www.104.com.tw/job/?jobno=5aerm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5aet6&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=59stz&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5bz4y&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=59yne&jobsource=104_bank1&hotjob_chr=



https://www.104.com.tw/job/?jobno=4vc66&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4sbaq&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4mcxm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4xk31&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=53sgs&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4c05u&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=4hvh2&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4hvuq&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=45qmo&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=45yac&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4eysv&jobsource=104_bank1&hot

https://www.104.com.tw/job/?jobno=5dku4&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=529wc&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4plc4&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4mt5b&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4sezz&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=4g49o&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=49gk4&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=grbt&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=431hu&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=46htn&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=2t2v3&jobsource=104_bank1&hotjob_chr=




https://www.104.com.tw/job/?jobno=5ezj0&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5fdfv&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5fdgx&jobsource=104_bank1&hotjob_

https://www.104.com.tw/job/?jobno=5c1fj&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5f6ox&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5f6or&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4o4t5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4e7r9&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5c1fk&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=5gl9t&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5gl9s&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4cmxz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4ouev&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5fp2b&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5glf0&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3uob4&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4rjxi&jobsource=104_bank1&hotj

{'BASH': 3,
 'C': 120,
 'C#': 67,
 'C++': 61,
 'CSS': 94,
 'DELPHI': 1,
 'GO': 3,
 'HTML': 140,
 'JAVA': 117,
 'JAVASCRIPT': 268,
 'MYSQL': 73,
 'PERL': 8,
 'PHP': 92,
 'PYTHON': 39,
 'R': 11,
 'RUBY': 4,
 'SCALA': 9,
 'SQL': 95,
 'SWIFT': 13,
 'TYPESCRIPT': 1}